# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

## Abre el archivo de datos y mira la información general. 

Iniciamos con la importación de la librería Pandas, la cual nos apoyará con el análisis de los datos obtenidos. Posteriormente procedemos con la carga de la base de datos.

In [53]:
# Cargar todas las librerías
import pandas as pd

# Carga los datos
scores = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

A continución encontramos la dimensión de los datos y exploramos las primeras 20 filas en busqueda de algunos errores para ir planificando la limpieza de la información.

In [54]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos

scores.shape

(21525, 12)

In [55]:
# vamos a mostrar las primeras filas N

scores.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


De la muestra de datos podemos observar varios problemas que habrá que atender de los datos como son:

- Días trabajados negativos en una gran cantidad de casos, además de contar con decimales innecesarios.
- Entradas idénticas, pero con mayúsculas y minúsculas en la columna 'education' que habrá que homologar.
- Información ausente respecto a los días de trabajo y nivel de ingresos.
- En la columna de propósito igual será necesario homologar las entradas ya que contamos con algunas diferentes que hacern referencia a la misma clasificación como "having a wedding" y "to have a wedding".
- Los ingresos cuentan con tres decimales, sobrando uno.

In [56]:
# Obtener información sobre los datos
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Rectificamos que los valores ausentes se encuentran en las columnas de 'days_employed' y 'total income'. Además 'education_id' y 'family_status_id' podrían convertirse en cadenas para evitar cálculos matemáticos con estas columnas.

In [57]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos

scores.loc[scores['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Con este resumen de las filas con datos ausentes en la columna 'days employed', parece existir una coincidencia con los valores faltantes en la columna 'total income'. Sin embargo para confirmar esta teoría, que además coincide por el número de datos faltante, hará falta revisar un filtrado en el que se busque la intersección de valores ausentes en ambas columnas.

In [58]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
scores.loc[(scores['days_employed'].isna())&(scores['total_income'].isna())]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Conclusión intermedia**

Con este último filtro, dado que la cantidad de filas en la tabla concide con los valores faltantes en ambas columnas, podemos confirmar que cuando faltan los valores en una columna, también faltan en la otra. Esto puede deberse a algún error de sistema al momento de cargar los datos en la base.

Las 2174 filas con información faltante corresponden al 10.1% de los datos, lo cual es bastante significativo, por lo que se procederá a completar los valores ausentes.

Al revisar la muestra de datos, parece existir una importante correlación entre la falta de datos y el nivel educativo, ya que la mayoría parece contar con la educación secundaria como su nivel de estudios.

In [59]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
scores.loc[(scores['days_employed'].isna())&(scores['education_id']==1)]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21426,0,NaN,49,secondary education,1,married,0,F,employee,1,NaN,property
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [60]:
# Comprobación de la distribución
scores.loc[scores['days_employed'].isna()].groupby('education_id')['education'].count()


education_id
0     544
1    1540
2      69
3      21
Name: education, dtype: int64

Con esta clasificación por nivel educativo, podemos encontrar una fuerte correlación con la falta de valores en aquellos individuos que se encuentran en un nivel de 'educación secundaria'.

**Posibles razones por las que hay valores ausentes en los datos**

Es posible que esta relación recaiga en algún error de sistema generado al elegir el valor '1' de 'education_id'.

In [61]:
# Comprobando la distribución en el conjunto de datos entero

relation_ed=scores.groupby('education_id')['days_employed'].apply(lambda x: x.isnull().sum() / len(x)).to_frame(name='ratio')
relation_ed['count']=scores.groupby('education_id')['days_employed'].apply(lambda x: len(x))

relation_ed.style.format({'ratio':'{:,.2%}'.format,})



,ratio,count
education_id,,
0,10.34%,5260
1,10.11%,15233
2,9.27%,744
3,7.45%,282
4,0.00%,6


**Conclusión intermedia**

La distribución parece terminar indicando cierta aleatoriedad ya que el 10.1% de las filas con valores faltantes es bastante similar a los porcentajes de valores faltantes en relación con el total por cada segmento de nivel de estudios, variable que parecía tener la mayor relación con los valores ausentes.

Para confirmar esta hipótesis, utilizaremos además la comparación relacionada con la existencia de una deuda con el pago de un préstamo, que sería la segunda variable que parece mantener mayor relación con los valores ausentes.


In [62]:
relation_debt=scores.groupby('debt')['days_employed'].apply(lambda x: x.isnull().sum() / len(x)).to_frame(name='ratio')
relation_debt['count']=scores.groupby('debt')['days_employed'].apply(lambda x: len(x))

relation_debt.style.format({'ratio':'{:,.2%}'.format,})


,ratio,count
debt,,
0,10.13%,19784
1,9.76%,1741


**Conclusión intermedia**

Como podemos observar, nuevamente se repite el patrón del 10.1%, fortaleciendo la teoría de que los huecos de información son aleatorios.

Finalmente repetiremos el mismo ejercicio con el resto de las variables: 'children', 'dob_years', 'family_status_id', 'gender' e 'income_type'

In [63]:
# Comprobación de otros patrones: explica cuáles

relation_children=scores.groupby('children')['days_employed'].apply(lambda x: x.isnull().sum() / len(x)).to_frame(name='ratio')
relation_children['count']=scores.groupby('children')['days_employed'].apply(lambda x: len(x))

print(relation_children)

print()

relation_dob_years=scores.groupby('dob_years')['days_employed'].apply(lambda x: x.isnull().sum() / len(x)).to_frame(name='ratio')
relation_dob_years['count']=scores.groupby('dob_years')['days_employed'].apply(lambda x: len(x))

print(relation_dob_years)

print()

relation_family_status_id=scores.groupby('family_status_id')['days_employed'].apply(lambda x: x.isnull().sum() / len(x)).to_frame(name='ratio')
relation_family_status_id['count']=scores.groupby('family_status_id')['days_employed'].apply(lambda x: len(x))

print(relation_family_status_id)

print()

relation_gender=scores.groupby('gender')['days_employed'].apply(lambda x: x.isnull().sum() / len(x)).to_frame(name='ratio')
relation_gender['count']=scores.groupby('gender')['days_employed'].apply(lambda x: len(x))

print(relation_gender)

print()

relation_income_type=scores.groupby('income_type')['days_employed'].apply(lambda x: x.isnull().sum() / len(x)).to_frame(name='ratio')
relation_income_type['count']=scores.groupby('income_type')['days_employed'].apply(lambda x: len(x))

relation_income_type




             ratio  count
children                 
-1        0.063830     47
 0        0.101703  14149
 1        0.098589   4818
 2        0.099270   2055
 3        0.109091    330
 4        0.170732     41
 5        0.111111      9
 20       0.118421     76

              ratio  count
dob_years                 
0          0.099010    101
19         0.071429     14
20         0.098039     51
21         0.162162    111
22         0.092896    183
23         0.141732    254
24         0.079545    264
25         0.064426    357
26         0.085784    408
27         0.073022    493
28         0.113320    503
29         0.091743    545
30         0.107407    540
31         0.116071    560
32         0.072549    510
33         0.087780    581
34         0.114428    603
35         0.103728    617
36         0.113514    555
37         0.098696    537
38         0.090301    598
39         0.089005    573
40         0.108374    609
41         0.097199    607
42         0.108878    597
43        

,ratio,count
income_type,,
business,0.099902,5085
civil servant,0.100754,1459
employee,0.099379,11119
entrepreneur,0.500000,2
paternity / maternity leave,0.000000,1
retiree,0.107106,3856
student,0.000000,1
unemployed,0.000000,2


**Conclusiones**

En general seguimos comprobando que la proporción de valores faltantes respecto al total considerando las divisiones de los distintos grupos, todas tienden al 10.1, en especial cuando la cantidad de valores en la categoría es mayor. Como consecuencia podemos asumir que en efecto se trata de una aleatoriedad.

El valor más dispar a la relación de 10.1% es el correspondiente a los emprendedores, pero al considerar que sólo existen dos registros en esta categoría, podemos omitirla, como razón de peso para la falta de datos.

Estos valores ausentes, dada su naturaleza aleatoria vamos a poder abordarlos sustituyéndolos por media o mediana de la información agrupada por distintos conceptos. Al tratarse de las variables correspondientes a los días trabajados y los ingresos totales, los factores que más podrían afectar a estas variables serían edad, género y el tipo de ingreso. Evaluaremos la consideración de cada una de estas variables.

A continuación, para transformar los datos, se estudiarán los duplicados en los registros, los cuales, al no existir un identificador de cliente, dificilmente podremos eliminar, ya que puede haber clientes que generen el mismo tipo de información.

Posteriormente se buscará homogeneizar la información de las columnas 'education' y 'purpose' con el propósito de que sea más manejable, ya que esta viene con diferencias en su forma de escribir distintos elementos de una misma categoría.

Finalmente procederemos a evaluar la influencia de variables mencionadas previamente sobre las que tienen valores faltantes, una vez elegido el material a utilizar, se considerará si es la media o la mediana la que resultaría más apropiada a considerar, y finalmente se rellenarán los valores ausentes.

## Transformación de datos


In [64]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente

sorted(scores['education'].unique())

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [65]:
# Podemos observar que será necesario corregir varios de estos valores. Primero observemos si existe una relación directa inequívoca con la columna education_id y después corrijamos de acuerdo con el resultado.

print('0')
print(scores.loc[scores['education_id']==0]['education'].unique())
print()
print('1')
print(scores.loc[scores['education_id']==1]['education'].unique())
print()
print('2')
print(scores.loc[scores['education_id']==2]['education'].unique())
print()
print('3')
print(scores.loc[scores['education_id']==3]['education'].unique())
print()
print('4')
print(scores.loc[scores['education_id']==4]['education'].unique())
print()

scores.loc[scores['education_id']==0,'education']="bachelor's degree"
scores.loc[scores['education_id']==1,'education']='secondary education'
scores.loc[scores['education_id']==2,'education']='some college'
scores.loc[scores['education_id']==3,'education']='primary education'
scores.loc[scores['education_id']==4,'education']='Graduate Degree'

0
["bachelor's degree" "BACHELOR'S DEGREE" "Bachelor's Degree"]

1
['secondary education' 'Secondary Education' 'SECONDARY EDUCATION']

2
['some college' 'SOME COLLEGE' 'Some College']

3
['primary education' 'PRIMARY EDUCATION' 'Primary Education']

4
['Graduate Degree' 'GRADUATE DEGREE' 'graduate degree']



In [66]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido

sorted(scores['education'].unique())

['Graduate Degree',
 "bachelor's degree",
 'primary education',
 'secondary education',
 'some college']

[Comprueba los datos de la columna `children`]

In [67]:
# Veamos la distribución de los valores en la columna `children`
sorted(scores['children'].unique())

[-1, 0, 1, 2, 3, 4, 5, 20]

En esta columna encontramos datos muy extraños, como -1 hijos o 20 hijos. Para evaluar cómo procederemos con estos datos, lo primero a realizar será considerar el volumen que representa cada uno.

In [68]:
# Obtenemos el volumen de los datos, para considerar su impacto en los datos:

wrong_value_count0=scores.loc[scores['children']==-1]['children'].count()+scores.loc[scores['children']==20]['children'].count()



print("{:.2%}".format(wrong_value_count0/scores['days_employed'].count()))

print(scores['children'].value_counts())

# Considerando el pequeño volumen podríamos asumir que se trata de errores de captura, y procederemos a cambiarlos por 1 y 2, como corresponda con su escritura más aproximada.

scores.loc[scores['children']==-1,'children']=1
scores.loc[scores['children']==20,'children']=2


0.64%
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [69]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado

print(scores['children'].value_counts())


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


En la respuesta anterior, podemos confirmar que los valores para children quedaron arreglados, y además responden decresiendo en el conteo conforme aumenta el número de hijos.

A continuación estudiaremos la columna 'days_employed', la cual nos interesa que nos muestre los días, sin fracciones, siempre debe tratarse de valores positivos y valores menores a la vida del individuo. Calcularemos el porcentaje de los últimos dos errores, los solucionaremos y corregiremos el formato.

In [70]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
wrong_value_count = scores.loc[scores['days_employed']<0]['days_employed'].count()+scores.loc[scores['days_employed']>(scores['dob_years']*365)]['days_employed'].count()
print("{:.0%}".format(wrong_value_count/scores['days_employed'].count()))

100%


Observamos que la totalidad de los valores de la columna es erroneo, o son negativos o exceden considerablemente la vida del individuo. Estos errores deben haberse debido a problemas técnicos, teoría que es apoyada por el hecho de que todos los números vienen fraccionados, lo que indica que son valores obtenidos como algún cálculo computacional.

Analizándo la información, parece ser que los valores introducidos como negativos, en caso de tomarlos como positivos núnca sobrepasan la edad del individuo, por lo que se implementará esta corrección en la información. Por el otro lado, los valores positivos cobran sentido al considerarse como horas trabajadas y no días trabajados, por lo que igualmente se trabajarán estos datos, para obtener información más cercana con la realidad.

In [71]:
# Aborda los valores problemáticos, si existen.

def correccion_dias_empleado(days):
    if pd.isna(days):
        return float('nan')
    elif days<0:
        days = -days
        return int(days)
    else:
        days = days/24
        return int(days)
        
print(correccion_dias_empleado(-8437.673028))
print(correccion_dias_empleado(340266.072047))

scores['days_employed']=scores['days_employed'].apply(correccion_dias_empleado)


8437
14177


In [72]:
# Comprueba el resultado - asegúrate de que esté arreglado
wrong_value_count2 = scores.loc[scores['days_employed']<0]['days_employed'].count()+scores.loc[scores['days_employed']>(scores['dob_years']*365)]['days_employed'].count()
print("{:.2%}".format(wrong_value_count2/scores['days_employed'].count()))
print()
scores.loc[scores['days_employed']>(scores['dob_years']*365)]






0.75%



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.0,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.0,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
157,0,14517.0,38,secondary education,1,married,0,F,retiree,1,18169.704,purchase of a car
270,3,1872.0,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,16577.0,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
...,...,...,...,...,...,...,...,...,...,...,...,...
20560,0,13891.0,33,secondary education,1,married,0,M,retiree,0,26145.901,education
20577,0,13822.0,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
20829,1,16591.0,40,secondary education,1,married,0,F,retiree,0,6257.265,getting an education
21179,2,108.0,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


Con las soluciones implementadas descubrimos que ahora sólo el 0.75% de los datos tienen conflictos. Investigando en los errores restantes comprobamos que una gran cantidad tienen un problema con la edad, la cual marca 0.

Ahora echemos un vistazo a la edad de clientes para ver si hay algún problema allí. De entrada los ya identificados como 0, así como cualquier negativo o demasiado alto, no deberían considerarse valores correctos.

In [73]:
# Revisión de `dob_years` en busca de valores sospechosos y cuenta el porcentaje

print(sorted(scores['dob_years'].unique()))

wrong_value_count3=scores.loc[scores['dob_years']==0,'dob_years'].count()
print()
print("{:.2%}".format(wrong_value_count3/scores['dob_years'].count()))

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]

0.47%


Dado que el único error encontrado es el valor de cero, y considerando que corresponde a un porcentaje muy pequeño de los datos, además de que coincide y provoca una gran catidad de los errores que permanecen en la columna 'days_employed', estos datos serán omitidos en el resto del análisis. Si el porcentaje restante de 'days_employed' igualmente resulta despreciable, procederemos con su omisión.

In [74]:
# Resuelve los problemas en la columna `dob_years`, si existen
scores = scores.loc[scores['dob_years']!=0]

In [75]:
# Comprueba el resultado - asegúrate de que esté arreglado
print(sorted(scores['dob_years'].unique()))

wrong_value_count5=scores.loc[scores['dob_years']==0,'dob_years'].count()
print('Porcentaje errores edad')
      
print("{:.2%}".format(wrong_value_count5/scores['dob_years'].count()))

print('Porcentaje errores días empleado')
wrong_value_count4 = scores.loc[scores['days_employed']<0]['days_employed'].count()+scores.loc[scores['days_employed']>(scores['dob_years']*365)]['days_employed'].count()
print("{:.2%}".format(wrong_value_count4/scores['days_employed'].count()))

scores = scores.loc[(scores['days_employed']<(scores['dob_years']*365))|(scores['days_employed'].isna())]

print('Porcentaje errores días empleado')
wrong_value_count6 = scores.loc[scores['days_employed']<0]['days_employed'].count()+scores.loc[scores['days_employed']>(scores['dob_years']*365)]['days_employed'].count()
print("{:.2%}".format(wrong_value_count6/scores['days_employed'].count()))

[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
Porcentaje errores edad
0.00%
Porcentaje errores días empleado
0.28%
Porcentaje errores días empleado
0.00%


A continuación revisaremos la columna 'family_status' en sus valores únicos para encontrar anomalías.

In [76]:
# Veamos los valores de la columna 'family_status'

sorted(scores['family_status'].unique())

['civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower']

Comprobamos que los valores son correctos, no existe necesidad de modificar esta columna.

A continuación revisaremos la columna 'gender' en sus valores únicos para encontrar anomalías. De existir, serán trabajadas.

In [77]:
# Veamos los valores en la columna
sorted(scores['gender'].unique())

['F', 'M', 'XNA']

In [78]:
# Aborda los valores problemáticos, si existen
print(scores.loc[scores['gender']=='XNA'].count())

scores = scores.loc[scores['gender']!='XNA']

children            1
days_employed       1
dob_years           1
education           1
education_id        1
family_status       1
family_status_id    1
gender              1
income_type         1
debt                1
total_income        1
purpose             1
dtype: int64


In [79]:
# Comprueba el resultado - asegúrate de que esté arreglado

sorted(scores['gender'].unique())

['F', 'M']

A continuación revisaremos la columna 'income_type' en sus valores únicos para encontrar anomalías. De existir, serán trabajadas.

In [80]:
# Veamos los valores en la columna
sorted(scores['income_type'].unique())

['business',
 'civil servant',
 'employee',
 'entrepreneur',
 'paternity / maternity leave',
 'retiree',
 'student']

Comprobamos que los valores son correctos, no existe necesidad de modificar esta columna.


A continuación revisaremos la columna 'purpose' en sus valores únicos para encontrar anomalías. De existir, serán trabajadas.

In [81]:
sorted(scores['purpose'].unique())

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

Podemos encontrar una gran variedad de categorías que podríamos considerar la misma, por lo que habrá que renombrar la mayoría. Las dividiremos en grandes clasificaciones: 'car', 'wedding', 'real estate' and 'education'.

In [82]:
def clasificar(purpose):
    var1='car'
    var2='wedding'
    var3='educa'
    var4='hous'
    var5='real estate'
    var6= str(purpose)
    var7='property'
    var8='univer'
    if var1 in var6:
        return 'car'
    if var2 in var6:
        return 'wedding'
    if var3 in var6:
        return 'education'
    if var4 in var6:
        return 'real estate'
    if var5 in var6:
        return 'real estate'
    if var7 in var6:
        return 'real estate'
    if var8 in var6:
        return 'education'
    return var6
    
scores['purpose']=scores['purpose'].apply(clasificar)



In [83]:
sorted(scores['purpose'].unique())

['car', 'education', 'real estate', 'wedding']

Ahora revisemos la existencia de duplicados y evaluemos cómo proceder con ellos.

In [84]:
# Comprobar los duplicados

print(scores.duplicated().sum())

scores.loc[scores['days_employed'].isna()].duplicated().sum()

404


404

No existen filas duplicadas además de las que contienen valores vacíos, pero de existir no sería prudente eliminarlas, ya que al no existir un id de usuario, se puede asumir que existen varias personas con características iguales dentro de las variables presentadas en la tabla.

El conjunto de datos ha sido limpiado en aspectos de información incierta, valores similares en columnas y eliminación de filas con valores erroneos.

Las modificaciones han sido las siguientes, con su correspondiente porcentaje de datos:
-Eliminación de valores erroneos en edad y timpo empleado: 0.75%
-Corrección de nomenclatura en 'education' y purpose. (porcentaje indiferente ya que práctiamente todas se reclasificaron)
-Corrección de valores de children: 0.64%

Podemos observar que la tabla fue modificada en valores bastante poco en porcentaje.

# Trabajar con valores ausentes

Para la simplificación del trabajo resultará útil trabajar con los valores indexados de educación y familia, evitando tener que introducir cadenas extensas y más propensas al error humano.

In [85]:
# Encuentra los diccionarios
dir_edu = {0:"bachelor's degree",1:'secondary education',2:'some college', 3:'primary education', 4:'Graduate Degree'}
dir_fam = {0:'married',1:'´civil partnership',2:'widow / widower', 3:'divorced', 4:'unmarried'}

### Restaurar valores ausentes en `total_income`

Como se comentó al inicio, existen valores ausentes en las columnas de ingreso total y días trabajados, los cuales coninciden en ambas. Y comenzaremos tratando de obtener los valores ausentes de la columa ingreso total. Para esto será necesario evaluar el efecto de distintas variables sobre esta para poder determinar el método más correcto para sustituirla.

Comenzaremos creando una clasificación de edad, la cual será agrupando a los individuos en grupos por 10 años de edad.

In [86]:
# Vamos a escribir una función que calcule la categoría de edad

def age_group(age):
    if age <= 30:
        return '20-30'
    if age <= 40:
        return '31-40'
    if age <= 50:
        return '41-50'
    if age <= 60:
        return '51-60'
    if age <= 70:
        return '61-70'
    if age <= 80:
        return '71-80'

In [87]:
# Prueba si la función funciona bien
print(age_group(27))
print(age_group(52))
print(age_group(78))

20-30
51-60
71-80


In [88]:
# Crear una nueva columna basada en la función

scores['age_group'] = scores['dob_years'].apply(age_group)

In [89]:
# Comprobar cómo los valores en la nueva columna

scores.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,real estate,41-50
1,1,4024.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,31-40
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,real estate,31-40
3,3,4124.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,31-40
4,0,14177.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,51-60
5,0,926.0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,real estate,20-30
6,0,2879.0,43,bachelor's degree,0,married,0,F,business,0,38484.156,real estate,41-50
7,0,152.0,50,secondary education,1,married,0,M,employee,0,21731.829,education,41-50
8,2,6929.0,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,31-40
9,0,2188.0,41,secondary education,1,married,0,M,employee,0,23108.150,real estate,41-50


Como se mencionó anteriormente se estima que el impacto en la variable de ingreso total debe verse mayormente ligado a las variables de edad, tipo de ingreso y educación. Para evaluar estos impactos se creará una nueva tabla sin valores vacíos sobre la que se verá el impacto de estas variables.

In [90]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien

scores_not_na=scores.loc[scores['days_employed'].isna()==False]

scores_not_na.head(20)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,real estate,41-50
1,1,4024.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,31-40
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,real estate,31-40
3,3,4124.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,31-40
4,0,14177.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,51-60
5,0,926.0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,real estate,20-30
6,0,2879.0,43,bachelor's degree,0,married,0,F,business,0,38484.156,real estate,41-50
7,0,152.0,50,secondary education,1,married,0,M,employee,0,21731.829,education,41-50
8,2,6929.0,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,31-40
9,0,2188.0,41,secondary education,1,married,0,M,employee,0,23108.150,real estate,41-50


In [91]:
# Examina los valores medios de los ingresos en función de los factores que identificaste

print(scores_not_na.pivot_table(index='age_group',values='total_income'))
print()
print(scores_not_na.pivot_table(index='education',values='total_income'))
print()
print(scores_not_na.pivot_table(index='income_type',values='total_income'))

           total_income
age_group              
20-30      25838.430041
31-40      28417.824539
41-50      28386.234907
51-60      25482.856294
61-70      23245.390243
71-80      19575.454327

                     total_income
education                        
Graduate Degree      27960.024667
bachelor's degree    33156.584261
primary education    21169.373448
secondary education  24617.200041
some college         29035.057865

                             total_income
income_type                              
business                     32397.307219
civil servant                27361.316126
employee                     25824.679592
entrepreneur                 79866.103000
paternity / maternity leave   8612.661000
retiree                      21920.621014
student                      15712.260000


In [92]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
print(scores_not_na.pivot_table(index='age_group',values='total_income', aggfunc='median'))
print()
print(scores_not_na.pivot_table(index='education',values='total_income', aggfunc='median'))
print()
print(scores_not_na.pivot_table(index='income_type',values='total_income', aggfunc='median'))

           total_income
age_group              
20-30        22967.8075
31-40        24870.5560
41-50        24569.9680
51-60        22056.7710
61-70        19705.8550
71-80        18611.5935

                     total_income
education                        
Graduate Degree        25161.5835
bachelor's degree      28041.5435
primary education      18741.9760
secondary education    21854.8445
some college           25608.7945

                             total_income
income_type                              
business                       27563.0285
civil servant                  24083.5065
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18989.2090
student                        15712.2600


Podemos encontrar mucha discrepancia entre los valores medios y medianos en los tres aspectos. Esto puede deberse a los altos incrementos en puestos de alta jerarquía, que difieren mucho de los puestos más comunes. Por esto mismo es que para los cálculos se utilizarán valores medianos, que representan un ingreso más común.

Así mismo observamos que sin considerar los grupos de edad más avanzada que dependen principalmente de apoyos y jubilaciones, que la edad no es un factor tan determinante. Al contrario el género y el tipo de ingreso son más determinantes de una diferencia de ingresos.

Para la función que determinará el ingreso total se terminará considerando un promedio de las medianas correspondientes a estos tres factores para cada individuo.

In [93]:
#  Escribe una función que usaremos para completar los valores ausentes
        
def total_income_filler(row):
    age=row['age_group']
    ed=row['education']
    inc=row['income_type']
    if pd.isna(row['total_income']):
        try:
            det = scores_not_na.loc[(scores_not_na['age_group']==age)&(scores_not_na['education']==ed)&(scores_not_na['income_type']==inc)]['total_income'].median()
        except:
            det = float('nan')
        return det
    else:
        return row['total_income']

In [94]:
# Comprueba si funciona

test_table_data={'age_group':['20-30','61-70','51-60'],'education':['Graduate Degree',"bachelor's degree",'some college'],'income_type':['employee','retiree','employee'],'total_income':[float('nan'),float('nan'),5]}
test_table = pd.DataFrame(test_table_data)

print(total_income_filler(test_table.iloc[0]))
print()
print(total_income_filler(test_table.iloc[1]))
print()
print(total_income_filler(test_table.iloc[2]))
print()

test_table['total_income']=test_table.apply(total_income_filler,axis=1)

print(test_table)


nan

23030.247000000007

5.0

  age_group          education income_type  total_income
0     20-30    Graduate Degree    employee           NaN
1     61-70  bachelor's degree     retiree     23030.247
2     51-60       some college    employee         5.000


In [95]:
# Aplícalo a cada fila
scores['total_income']=scores.apply(total_income_filler,axis=1)

In [96]:
# Comprueba si tenemos algún error

scores.info()

print(scores.head(15))
print()
print(scores.groupby('age_group')['total_income'].median())
print()
print(scores.groupby('education')['total_income'].median())
print()
print(scores.groupby('income_type')['total_income'].median())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21369 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21369 non-null  int64  
 1   days_employed     19205 non-null  float64
 2   dob_years         21369 non-null  int64  
 3   education         21369 non-null  object 
 4   education_id      21369 non-null  int64  
 5   family_status     21369 non-null  object 
 6   family_status_id  21369 non-null  int64  
 7   gender            21369 non-null  object 
 8   income_type       21369 non-null  object 
 9   debt              21369 non-null  int64  
 10  total_income      21365 non-null  float64
 11  purpose           21369 non-null  object 
 12  age_group         21369 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB
    children  days_employed  dob_years            education  education_id  \
0          1         8437.0         42    bache

Los datos han sido sustituidos, comprobamos que ya sólo existen 4 valores vacíos, los cuales serán omitidos dado el bajo porcentaje y que calcular con menos variables sería menos fiable. Los valores ya existentes se conservaron y que aunque las medianas sí llegaron a variar un poco, no provocaron cambios drásticos en las mediciones.

In [97]:
scores=scores.dropna(subset=['total_income'])

scores.info()

print(scores.loc[scores['total_income'].isna()])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21365 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21365 non-null  int64  
 1   days_employed     19205 non-null  float64
 2   dob_years         21365 non-null  int64  
 3   education         21365 non-null  object 
 4   education_id      21365 non-null  int64  
 5   family_status     21365 non-null  object 
 6   family_status_id  21365 non-null  int64  
 7   gender            21365 non-null  object 
 8   income_type       21365 non-null  object 
 9   debt              21365 non-null  int64  
 10  total_income      21365 non-null  float64
 11  purpose           21365 non-null  object 
 12  age_group         21365 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gend

###  Restaurar valores en `days_employed`

Para llenar los valores vacíos respecto a días empleado, se considerarán igualmente los valores de edad, tipo de ingreso y género, pero además se considerará el factor de número de hijos y nivel de estudios; esto debido a que el tiempo de estudios y de cuidado de los hijos pudieron afectar el inicio de su vida laboral. Procederemos a considerar el impacto de las medias y medianas de estas variables.

In [98]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

print(scores_not_na.pivot_table(index='age_group', values='days_employed',aggfunc='median'))
print()
print(scores_not_na.pivot_table(index='gender', values='days_employed',aggfunc='median'))
print()
print(scores_not_na.pivot_table(index='income_type', values='days_employed',aggfunc='median'))
print()
print(scores_not_na.pivot_table(index='children', values='days_employed',aggfunc='median'))
print()
print(scores_not_na.pivot_table(index='education_id', values='days_employed',aggfunc='median'))


           days_employed
age_group               
20-30             1043.0
31-40             1613.0
41-50             2199.0
51-60             6481.0
61-70            14844.0
71-80            15006.5

        days_employed
gender               
F              2534.0
M              1659.0

                             days_employed
income_type                               
business                            1547.5
civil servant                       2672.5
employee                            1575.5
entrepreneur                         520.0
paternity / maternity leave         3296.0
retiree                            15206.5
student                              578.0

          days_employed
children               
0                2612.5
1                1666.0
2                1677.5
3                1733.5
4                1905.5
5                1231.5

              days_employed
education_id               
0                    1891.5
1                    2383.5
2                

In [99]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
print(scores_not_na.pivot_table(index='age_group', values='days_employed',aggfunc='mean'))
print()
print(scores_not_na.pivot_table(index='gender', values='days_employed',aggfunc='mean'))
print()
print(scores_not_na.pivot_table(index='income_type', values='days_employed',aggfunc='mean'))
print()
print(scores_not_na.pivot_table(index='children', values='days_employed',aggfunc='mean'))
print()
print(scores_not_na.pivot_table(index='education_id', values='days_employed',aggfunc='mean'))

           days_employed
age_group               
20-30        1278.858547
31-40        2094.910613
41-50        3252.854492
51-60        8237.764125
61-70       12823.513469
71-80       13900.989796

        days_employed
gender               
F         5289.709749
M         3298.547274

                             days_employed
income_type                               
business                       2112.251865
civil servant                  3388.011485
employee                       2328.101666
entrepreneur                    520.000000
paternity / maternity leave    3296.000000
retiree                       15204.134262
student                         578.000000

          days_employed
children               
0           5613.690404
1           2902.373080
2           2267.288947
3           2393.777397
4           2604.764706
5           1432.000000

              days_employed
education_id               
0               3679.525674
1               5007.867809
2               2

Dado lo anterior encontramos nuevamente una diferencia importante entre medias y medianas tanto para el género y la edad, pero que mantienen una escalabilidad en cuanto al número de años vividos y la diferencia de la situación laboral de hombres y mujeres. En cuanto al tipo de ingreso podemos observar que media y mediana es prácticamente igual. Y donde encontramos unos resultados algo aleatorios (se tiene en consideración que el education ID no está ordenado), por lo que dejaremos fuera a estas variables.

Nos concentraremos en las medianas, dado que son más representativas de la sociedad en general, como se realizó con la otra columna.

In [100]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
def days_employed_filler(row):
    age=row['age_group']
    gen=row['gender']
    inc=row['income_type']
    if pd.isna(row['days_employed']):
        try:
            det=scores_not_na.loc[(scores_not_na['age_group']==age)&(scores_not_na['gender']==gen)&(scores_not_na['income_type']==inc)]['days_employed'].median()
        except:
            det = float('nan')
        return det
    else:
        return row['days_employed']

In [101]:
# Comprueba que la función funciona

test_table_data_2={'age_group':['20-30','61-70','51-60'],'gender':['M','F','M'],'income_type':['employee','retiree','employee'],'days_employed':[float('nan'),float('nan'),7000]}
test_table_2 = pd.DataFrame(test_table_data_2)

print(days_employed_filler(test_table_2.iloc[0]))
print()
print(days_employed_filler(test_table_2.iloc[1]))
print()
print(days_employed_filler(test_table_2.iloc[2]))
print()

test_table_2['days_employed']=test_table_2.apply(days_employed_filler,axis=1)

print(test_table)

971.0

15275.0

7000.0

  age_group          education income_type  total_income
0     20-30    Graduate Degree    employee           NaN
1     61-70  bachelor's degree     retiree     23030.247
2     51-60       some college    employee         5.000


In [102]:
# Aplicar la función al income_type

scores['days_employed']=scores.apply(days_employed_filler,axis=1)

In [103]:
# Comprueba si la función funcionó

scores.info()

print(scores.head(15))
print()
print(scores.groupby('age_group')['total_income'].median())
print()
print(scores.groupby('gender')['total_income'].median())
print()
print(scores.groupby('income_type')['total_income'].median())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21365 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21365 non-null  int64  
 1   days_employed     21365 non-null  float64
 2   dob_years         21365 non-null  int64  
 3   education         21365 non-null  object 
 4   education_id      21365 non-null  int64  
 5   family_status     21365 non-null  object 
 6   family_status_id  21365 non-null  int64  
 7   gender            21365 non-null  object 
 8   income_type       21365 non-null  object 
 9   debt              21365 non-null  int64  
 10  total_income      21365 non-null  float64
 11  purpose           21365 non-null  object 
 12  age_group         21365 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB
    children  days_employed  dob_years            education  education_id  \
0          1         8437.0         42    bache

Como podemos observar, todos los valores ausentes han sido llenados con información relativa a las medianass que nos apoyará con el análisis.

## Clasificación de datos

A continuación crearemos una nueva clasificación respecto al nivel de ingresos.

In [104]:
# Muestra los valores de los datos seleccionados para la clasificación

scores['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21365, dtype: float64

Comrpobemos valores únicos.

In [105]:
print(scores['total_income'].min())
print(scores['total_income'].max())
sorted(scores['total_income'].unique())

3306.762
362496.645


[3306.762,
 3418.824,
 3471.216,
 3503.298,
 3595.641,
 3815.153,
 3913.227,
 4036.463,
 4049.374,
 4212.77,
 4245.348,
 4386.4,
 4444.179,
 4465.254,
 4494.861,
 4592.45,
 4650.812,
 4664.644,
 4672.012,
 4708.271,
 4759.97,
 4812.103,
 4818.545999999999,
 4860.001,
 4919.749,
 5002.295,
 5028.623,
 5029.439,
 5037.321,
 5045.56,
 5053.838,
 5090.55,
 5112.186,
 5137.573,
 5148.514,
 5167.9940000000015,
 5168.082,
 5172.669,
 5195.285,
 5208.353,
 5217.0340000000015,
 5220.544,
 5259.254,
 5274.611,
 5288.165,
 5290.465,
 5330.769,
 5331.621,
 5335.014,
 5402.85,
 5409.738,
 5430.683000000001,
 5443.908,
 5452.4940000000015,
 5461.996,
 5464.092,
 5478.583000000001,
 5490.018,
 5496.834,
 5514.581,
 5515.539000000002,
 5529.334,
 5531.2040000000015,
 5562.874,
 5577.521,
 5579.965,
 5591.44,
 5604.991999999998,
 5622.0790000000015,
 5630.865,
 5639.846,
 5651.584,
 5703.853,
 5768.392,
 5772.8780000000015,
 5801.651,
 5803.271,
 5820.374,
 5826.733,
 5831.255,
 5837.099,
 5863.853,
 5

Podemos observar que el rango de valores es muy amplio, por lo que se considerará una división cada 20000 de ingreso, con el propósito de obtener 14 clasificaciones distintas.

In [106]:
# Escribamos una función para clasificar los datos en función de los grupos mencionados

def clasificacion_ingresos(monto):
    if monto<=20000:
        return '000000-20000'
    elif monto<=40000:
        return '020001-40000'
    elif monto<=60000:
        return '040001-60000'
    elif monto<=80000:
        return '060001-80000'
    elif monto<=100000:
        return '080001-100000'
    elif monto<=120000:
        return '100001-120000'
    elif monto<=140000:
        return '120001-140000'
    elif monto<=160000:
        return '140001-160000'
    elif monto<=180000:
        return '160001-180000'
    elif monto<=200000:
        return '180001-200000'
    elif monto<=220000:
        return '200001-220000'
    elif monto<=240000:
        return '220001-240000'
    elif monto<=260000:
        return '240001-260000'
    elif monto<=280000:
        return '260001-280000'
    elif monto<=300000:
        return '280001-300000'
    elif monto<=320000:
        return '300001-320000'
    elif monto<=340000:
        return '320001-340000'
    elif monto<=360000:
        return '340001-360000'
    elif monto<=380000:
        return '360001-380000'

In [107]:
# Comprobamos el funcionamiento de la función

print(clasificacion_ingresos(49000))
print(clasificacion_ingresos(111000))
print(clasificacion_ingresos(267000))


040001-60000
100001-120000
260001-280000


In [109]:
# Creamos nueva columna total_income_group

scores['total_income_group']=scores['total_income'].apply(clasificacion_ingresos)

In [110]:
# Verificar columna
scores.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_group
0,1,8437.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,real estate,41-50,040001-60000
1,1,4024.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,31-40,000000-20000
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,real estate,31-40,020001-40000
3,3,4124.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,31-40,040001-60000
4,0,14177.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,51-60,020001-40000
5,0,926.0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,real estate,20-30,040001-60000
6,0,2879.0,43,bachelor's degree,0,married,0,F,business,0,38484.156,real estate,41-50,020001-40000
7,0,152.0,50,secondary education,1,married,0,M,employee,0,21731.829,education,41-50,020001-40000
8,2,6929.0,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,31-40,000000-20000
9,0,2188.0,41,secondary education,1,married,0,M,employee,0,23108.150,real estate,41-50,020001-40000


## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [111]:
# Comprueba los datos sobre los hijos y los pagos puntuales
print(scores.groupby('children')['debt'].agg(['mean','median','sum','count']))
print()
##Calcular la tasa de incumplimiento en función del número de hijos
children_rate=scores.groupby('children')['debt'].sum()/scores.groupby('children')['debt'].count()
print(children_rate)

              mean  median   sum  count
children                               
0         0.075087       0  1054  14037
1         0.090740       0   439   4838
2         0.095125       0   201   2113
3         0.082569       0    27    327
4         0.097561       0     4     41
5         0.000000       0     0      9

children
0    0.075087
1    0.090740
2    0.095125
3    0.082569
4    0.097561
5    0.000000
Name: debt, dtype: float64


**Conclusión**

Podemos observar que la tasa y la media son iguales, ya que la deuda está marcada por un valor de 1. Encontramos que mientras siguen sindo relevantes los grupos, y no se vuelven la extrema minoría de 5 hijos, existe una tndencia de incumplimiento mayor de conforme incrementa el número de hijos.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [112]:
# Comprueba los datos del estado familiar y los pagos a tiempo
print(scores.groupby('family_status')['debt'].agg(['mean','median','sum','count']))
print()


# Calcular la tasa de incumplimiento basada en el estado familiar
family_rate=scores.groupby('family_status')['debt'].sum()/scores.groupby('family_status')['debt'].count()
print(family_rate)


                       mean  median  sum  count
family_status                                  
civil partnership  0.092646       0  383   4134
divorced           0.072034       0   85   1180
married            0.074992       0  923  12308
unmarried          0.097526       0  272   2789
widow / widower    0.064990       0   62    954

family_status
civil partnership    0.092646
divorced             0.072034
married              0.074992
unmarried            0.097526
widow / widower      0.064990
Name: debt, dtype: float64


**Conclusión**

Podemos considerar que todos los grupos tienen una población relevante, y que se detaca un incumplimiento por parte de los solteros o quienes tienen una unión civil, mientras que las personas que han vivido el matrimonio cumplen con más frecuencia, sobre todo quienes enviudaron. Esto último puede deberse a que el último grupo tiene una tendencia a ya no tener dependientes y pueden administrarse mejor.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [113]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
print(scores.groupby('total_income_group')['debt'].agg(['mean','median','sum','count']))
print()


# Calcular la tasa de incumplimiento basada en el nivel de ingresos
total_income_group_rate=scores.groupby('total_income_group')['debt'].sum()/scores.groupby('total_income_group')['debt'].count()
print(total_income_group_rate.sort_index)


                        mean  median  sum  count
total_income_group                              
000000-20000        0.082538       0  631   7645
020001-40000        0.082456       0  901  10927
040001-60000        0.072976       0  155   2124
060001-80000        0.053571       0   24    448
080001-100000       0.065574       0    8    122
100001-120000       0.040000       0    2     50
120001-140000       0.111111       0    2     18
140001-160000       0.000000       0    0      6
160001-180000       0.100000       0    1     10
180001-200000       0.000000       0    0      4
200001-220000       0.000000       0    0      3
220001-240000       0.000000       0    0      1
240001-260000       0.000000       0    0      2
260001-280000       0.000000       0    0      3
340001-360000       1.000000       1    1      1
360001-380000       0.000000       0    0      1

<bound method Series.sort_index of total_income_group
000000-20000     0.082538
020001-40000     0.082456
040001-6000

**Conclusión**

En general, sin considerar las tres claras excepciones, se puede notar una clara tendencia al cumplimiento con los préstamos conforme aumenta el nivel de ingresos. Las excepciones pueden deberse a los pocos individuos en el más alto expectro de ingresos. Esto tiene una clara explicación, ya que más rápidamente pueden cubrir sus deudas.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [114]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos

print(scores.groupby('purpose')['debt'].agg(['mean','median','sum','count']))

                 mean  median  sum  count
purpose                                  
car          0.092839       0  398   4287
education    0.091957       0  367   3991
real estate  0.072258       0  778  10767
wedding      0.078448       0  182   2320


**Conclusión**

Podemos observar que los préstamos inmobiliarios son los que menor tendencia tienen a caer en deudas, esto debe relacionarse directamente con los montos y la duración de los préstamos, ya que se trata de un gran compromiso por adquirir, mientras que los prestamos para carros y educación, que podrían adquirir personas en mayor necesidad, son los que tienen más tendencia a incumplirse.

# Conclusión general 

Tras un exhaustivo análisis donde tuvieron que completarse algunos datos considerando medianas de las principales cateogorías a las que pertenecían, que fueron los casos de los días trabajados y el ingreso total. Y en el que una corrección de términos y clasificación de datos fue necesaria como es el caso del nivel educativo y la razón del préstamo, se pudieron obtener las siguientes conclusiones.

*El aumento en el número de hijos tiende a incrementar el riesgo de caer en deuda.
*Quienes han estado casados tienden a cumplir más con sus pagos, en especial cuando se encuentran en estado de viudez.
*Entre mayor sea el ingreso total del individuo es más propenso a mantenerse fuera del incumplimiento.
*Los prestamos inmobiliarios con los que menos incumplimiento presentan, al contrario de los que corresponden a eduación o automóviles.